<a href="https://colab.research.google.com/github/hrnrhty/my-vae-nnabla/blob/main/step1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 - VAE の学習 (MNIST)

最初のステップでは、[nnabla-examples](https://github.com/sony/nnabla-examples) に含まれる VAE (Variational Auto Encoder) をそのまま実行し、学習を行います。学習に用いるデータセットには、手書き数字のデータセットである [MNIST](http://yann.lecun.com/exdb/mnist/) が使用されています。

このノートブックでは、学習時間短縮のため GPU アクセラレーションを有効にしています。もし、以降のコードがうまく動かなかった場合は、タブメニューの「ランタイム」 > 「ランタイムのタイプを変更」をクリックし、「ハードウェア アクセラレータ」のドロップダウンメニューで「GPU」が選択されているか確認してください。「GPU」が選択されていなかった場合は、「GPU」を選択して「保存」ボタンをクリックしてください。

## Google Drive のマウント

学習結果の永続化のため Google Drive をマウントします。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## NNabla のインストール

Sony の [Neural Network Libraries](https://github.com/sony/nnabla) (NNabla) は Google Colab のランタイムにインストールされていないため、インストールします。ここでは、GPU アクセラレーションが使用できるよう、CUDA 版をインストールします。

NNabla の依存パッケージの中には既に Google Colab にインストールされているものがあり、NNabla が要求するバージョンと一致していない場合インストール時にエラーが発生することがあります。エラーを無視して先に進んでも問題なく動作することが多いですが、ここではエラー発生を回避します。具体的には、仮想環境を作成し、その中に NNabla をインストールしていきます。

通常、仮想環境の作成には Python 3.3 以降標準パッケージに含まれるようになった `venv` が使用されますが、Google Colab ではうまく動作しません。そこで、代わりに [virtualenv](https://virtualenv.pypa.io/en/latest/) をインストールします。`virtualenv` は `venv` の元になったパッケージです。

まず、`virtualenv` をインストールし、続いて `venv` という名称の仮想環境を作成します。

In [2]:
!pip install virtualenv
!virtualenv venv

     |████████████████████████████████| 8.7 MB 4.4 MB/s 
     |████████████████████████████████| 461 kB 44.7 MB/s 
created virtual environment CPython3.7.12.final.0-64 in 1156ms
  creator CPython3Posix(dest=/content/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.0.4, setuptools==60.10.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


仮想環境をアクティベートし、仮想環境内に NNabla (CUDA版) をインストールします。2022 年 2 月 2 日時点、最新版である v1.25.0 をインストールします。

ここで、`!` によるシェルコマンドはサブプロセスとして実行されるため、コマンド実行終了後は状態が維持されないことに注意が必要です。`&&` で連結し1つのコマンドとしてシェルに渡しましょう。2 行に分けてしまうと、2 行目は仮想環境がアクティベートされていない別のサブプロセスになるため、NNabla が仮想環境外にインストールされてしまいます。

In [3]:
!source venv/bin/activate && pip install nnabla-ext-cuda100==1.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 KB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## nnabla-examples のクローン

[nnabla-examples](https://github.com/sony/nnabla-examples) は活発にメンテナンスされており、時々ディレクトリ構成が変更されるため、ここでは Tag v1.25.0 を指定してクローンします。

In [4]:
!if [ ! -d nnabla-examples ]; then git clone https://github.com/sony/nnabla-examples.git -b v1.25.0 --depth 1; fi

Cloning into 'nnabla-examples'...
remote: Enumerating objects: 1557, done.
remote: Counting objects: 100% (1557/1557), done.
remote: Compressing objects: 100% (1408/1408), done.
remote: Total 1557 (delta 241), reused 884 (delta 82), pack-reused 0
Receiving objects: 100% (1557/1557), 201.62 MiB | 14.05 MiB/s, done.
Resolving deltas: 100% (241/241), done.
Note: checking out '03b674a19e8289ce870cd3dfd2a0983d6ef004fc'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

Checking out files: 100% (1377/1377), done.


## 学習の実行

GPU アクセラレーションを有効にするため、`-c cudnn` オプションつきで `vae.py` を実行します。GPU を利用することで、学習にかかる時間が飛躍的に短縮されます。試しに CPU のみで実行したときの学習速度を体感してみたい場合は、`-c cudnn` を削除して実行してください。

また、ここでも、仮想環境のアクティベートを忘れないようにしましょう。

In [5]:
!source venv/bin/activate && cd nnabla-examples/image-classification/mnist-collection && python vae.py -c cudnn

2022-03-23 09:18:18,571 [nnabla][INFO]: Initializing CPU extension...
2022-03-23 09:18:18,973 [nnabla][INFO]: Running in cudnn
2022-03-23 09:18:19,586 [nnabla][INFO]: Initializing CUDA extension...
2022-03-23 09:18:19,649 [nnabla][INFO]: Initializing cuDNN extension...
2022-03-23 09:18:19,650 [nnabla][INFO]: DataSource with shuffle(True)
2022-03-23 09:18:19,650 [nnabla][INFO]: Getting label data from http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz.
train-labels-idx1-ubyte.gz: 100% 28.2k/28.2k [00:00<00:00, 18.6MB/s]
2022-03-23 09:18:19,724 [nnabla][INFO]: Getting label data done.
2022-03-23 09:18:19,724 [nnabla][INFO]: Getting image data from http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz.
train-images-idx3-ubyte.gz: 100% 9.45M/9.45M [00:00<00:00, 73.1MB/s]
2022-03-23 09:18:20,202 [nnabla][INFO]: Getting image data done.
2022-03-23 09:18:20,205 [nnabla][INFO]: Using DataIterator
2022-03-23 09:18:20,207 [nnabla][INFO]: DataSource with shuffle(True)
2022-03-23 09

## 学習結果を Google Drive に保存

In [6]:
!cd drive/MyDrive && if [ ! -d my-vae-nnabla ]; then mkdir my-vae-nnabla; fi
!cd drive/MyDrive/my-vae-nnabla && if [ -d step1 ]; then rm -rf step1; fi
!cp -r nnabla-examples/image-classification/mnist-collection/tmp.monitor.vae drive/MyDrive/my-vae-nnabla/step1

Google Drive に保存した学習結果は、Step 2 で使用します。